<a href="https://colab.research.google.com/github/chrisyandata/RAG_demo/blob/dev/Copy_of_Explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import numpy as np
import re
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

In [ ]:

try:
  print("loading model...")
  llm = pipeline("text-generation", model="gpt2", max_new_tokens=256)
  print("model loaded")
except:
  pass

loading model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


model loaded


In [ ]:
llm

TextGenerationPipeline: {'model': 'GPT2LMHeadModel', 'dtype': 'float32', 'device': 'cpu', 'input_modalities': 'text', 'output_modalities': ('text',)}

In [ ]:
docs = [
{"id": "1", "txt": "Python is a high-level, interpreted programming language. It was created by  Guido van Rossum and first released in 1991. Python emphasizes code readability with its  notable use of significant whitespace."},
{"id": "2", "txt": "FastAPI is a modern, fast web framework for building APIs with Python 3.7+.  It is based on standard Python type hints and provides automatic API documentation. FastAPI is  one of the fastest Python frameworks available."},
{"id": "3", "txt": "Machine learning is a subset of artificial intelligence that enables systems to  learn and improve from experience. It uses algorithms to parse data, learn from it, and make  predictions or decisions without being explicitly programmed."},
{"id": "4", "txt": "RAG (Retrieval-Augmented Generation) combines information retrieval with  text generation. It retrieves relevant documents from a knowledge base and uses them to  generate informed, contextual answers. This approach improves accuracy and reduces  hallucinations."},
{"id": "5", "txt": "Vector databases store data as high-dimensional vectors and enable efficient  similarity search. They are essential for modern AI applications, particularly in semantic search  and retrieval systems. Common examples include Pinecone, Weaviate, and Chroma."}
]

In [ ]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([x["txt"] for x in docs])


In [ ]:
retrieved_docs = []
answer = ""
problems = []
score = 1.0
is_valid = False
temp_storage = {}


In [ ]:
if len(sys.argv) < 2:
  print("usage: python minimal-rag.py <question> [-k NUM] [--json]")
  print("example: python minimal-rag.py 'What is RAG?' -k 2 --json")
  sys.exit(1)


In [ ]:
question = sys.argv[1]
k = 3
use_json = False
i = 2


In [ ]:
while i < len(sys.argv):
  if sys.argv[i] == "-k":
    k = int(sys.argv[i+1])
    i += 2
  elif sys.argv[i] == "--json":
    use_json = True
    i += 1
  else:
    i += 1


In [ ]:
# retrieved_docs = []
# answer = ""
# problems = []
# score = 1.0
temp_storage["question"] = question


In [ ]:

def get_data():
  global retrieved_docs, temp_storage
  query_vec = vectorizer.transform([question])
  similarities = cosine_similarity(query_vec, vectors)[0]
  indices = np.argsort(similarities)[-k:][::-1]
  temp_storage["sims"] = similarities
  for i in indices:
    if similarities[i] > 0:
      retrieved_docs.append({"doc": docs[i], "score": float(similarities[i])})
      if len(retrieved_docs) >= k:
          break

In [ ]:
get_data()

In [ ]:
if len(retrieved_docs) == 0:
  answer = "idk"
  problems.append("no docs")
  score = score * 0.4
  is_valid = False
  print("\nQ: {}".format(question))
  print(f"\nA: {answer}\n")
  print("Citations: None")
  print("\nCritique: ISSUES (confidence: {:.2f})".format(score))
  print("Problems: %s" % ', '.join(problems))
  if use_json:
    print(json.dumps({"answer": answer, "citations": retrieved_docs, "critique": {"ok": is_valid,  "score": score, "problems": problems}}, indent=2))
  sys.exit(0)


In [ ]:
def call_model():
  global answer, temp_storage
  ctx = ""
  counter = 0
  for r in retrieved_docs:
    counter += 1
    ctx += f"[{counter}] {r['doc']['txt']}\n\n"
  temp_storage["context"] = ctx
  prompt = "Context:\n{}\nQuestion: {}\n\nAnswer the question based only on the context  above. Cite sources using [1], [2], etc. Be concise.\n\nAnswer:".format(ctx,  temp_storage['question'])
  output = llm(prompt)
  answer = output[0]['generated_text'].split("Answer:")[-1].strip()
  if answer == "":
    answer = "idk"


In [ ]:
call_model()

In [ ]:
def validate_output():
  global problems, score, is_valid
  problems = []
  score = 1.0
  if "idk" in answer.lower():
    problems.append("no info")
    score = score * 0.5
  if len(answer) < 20:
      problems.append("short")
      score = score * 0.3
  refs = re.findall(r'\[(\d+)\]', answer)
  if len(retrieved_docs) > 0:
    if len(refs) == 0:
      problems.append("no refs")
      score = score * 0.7
  avg = 0
  for r in retrieved_docs:
    avg += r["score"]
  avg = avg / len(retrieved_docs)
  if avg < 0.1:
    problems.append("low relevance") score = score * 0.6
  if score > 0.6:
    if len(problems) < 3:
      is_valid = True
    else:
      is_valid = False
  else:
    is_valid = False
validate_output()


In [ ]:
print("\nQ: {}".format(question)) print(f"\nA: {answer}\n")
print("Citations:")
for r in retrieved_docs:
  doc_text = r['doc']['txt']
  if len(doc_text) > 80:
    doc_text = doc_text[:80] + "..."
print(" [{}] (score: {:.3f}) {}".format(r['doc']['id'], r['score'], doc_text))
if is_valid:
  print("\nCritique: OK (confidence: {:.2f})".format(score))
else:
  print(f"\nCritique: ISSUES (confidence: {score:.2f})")
if len(problems) > 0:
  print("Problems: %s" % ', '.join(problems))
result = {
"answer": answer,
"citations": retrieved_docs,
"critique": {"ok": is_valid, "score": score, "problems": problems}
}
if use_json:
  print(json.dumps(result, indent=2))
